In [5]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.functions import col, count,sum

In [6]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/29 17:12:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
print(spark.version)

3.4.1


In [8]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [16]:
joined_df = category_df.join(film_category_df, category_df.category_id == film_category_df.category_id)

result_df = joined_df.groupBy("name").agg(count("film_id").alias("count_films"))

sorted_result_df = result_df.orderBy(col("count_films").desc())

sorted_result_df.show()

+-----------+-----------+
|       name|count_films|
+-----------+-----------+
|     Sports|         74|
|    Foreign|         73|
|     Family|         69|
|Documentary|         68|
|  Animation|         66|
|     Action|         64|
|        New|         63|
|      Drama|         62|
|      Games|         61|
|     Sci-Fi|         61|
|   Children|         60|
|     Comedy|         58|
|     Travel|         57|
|   Classics|         57|
|     Horror|         56|
|      Music|         51|
+-----------+-----------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [10]:
actor_df = actor_df.alias('a')
film_actor_df = film_actor_df.alias('fa')
film_df = film_df.alias('f')
inventory_df = inventory_df.alias('i')
rental_df = rental_df.alias('r')

joined_df = actor_df \
    .join(film_actor_df, col('a.actor_id') == col('fa.actor_id')) \
    .join(film_df, col('fa.film_id') == col('f.film_id')) \
    .join(inventory_df, col('f.film_id') == col('i.film_id')) \
    .join(rental_df, col('i.inventory_id') == col('r.inventory_id'))

result_df = joined_df.groupBy(col('a.actor_id'), col('a.first_name'), col('a.last_name')) \
    .agg(count('r.rental_id').alias('rental_count'))

sorted_result_df = result_df.orderBy(col('rental_count').desc()).limit(10)

sorted_result_df.show()

+--------+----------+-----------+------------+
|actor_id|first_name|  last_name|rental_count|
+--------+----------+-----------+------------+
|     107|      GINA|  DEGENERES|         753|
|     181|   MATTHEW|     CARREY|         678|
|     198|      MARY|     KEITEL|         674|
|     144|    ANGELA|WITHERSPOON|         654|
|     102|    WALTER|       TORN|         640|
|      60|     HENRY|      BERRY|         612|
|     150|     JAYNE|      NOLTE|         611|
|      37|       VAL|     BOLGER|         605|
|      23|    SANDRA|     KILMER|         604|
|      90|      SEAN|    GUINESS|         599|
+--------+----------+-----------+------------+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [38]:
joined_df = payment_df \
    .join(rental_df, payment_df.rental_id == rental_df.rental_id) \
    .join(inventory_df, rental_df.inventory_id == inventory_df.inventory_id) \
    .join(film_category_df, inventory_df.film_id == film_category_df.film_id) \
    .join(category_df, film_category_df.category_id == category_df.category_id)

result_df = joined_df.groupBy("name") \
    .agg(sum("amount").alias("Summ"))

sorted_result_df = result_df.orderBy(col("Summ").desc()).limit(1)

sorted_result_df.show()

+------+-----------------+
|  name|             Summ|
+------+-----------------+
|Sports|5314.209999999848|
+------+-----------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [12]:
result_df = film_df.alias('f') \
    .join(inventory_df.alias('i'), col('f.film_id') == col('i.film_id'), how='left_anti') \
    .select('f.title')

result_df.show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [42]:
film_actor_df = film_actor_df.withColumnRenamed("actor_id", "fa_actor_id")
film_category_df = film_category_df.withColumnRenamed("film_id", "fc_film_id")
film_category_df = film_category_df.withColumnRenamed("category_id", "fc_category_id")

joined_df = actor_df \
    .join(film_actor_df, actor_df.actor_id == film_actor_df.fa_actor_id) \
    .join(film_category_df, film_actor_df.film_id == film_category_df.fc_film_id) \
    .join(category_df, film_category_df.fc_category_id == category_df.category_id)

filtered_df = joined_df.filter(category_df.name == 'Children')

result_df = filtered_df.groupBy("actor_id", "first_name", "last_name") \
    .agg(count("fa_actor_id").alias("film_count"))

sorted_result_df = result_df.orderBy(col("film_count").desc()).limit(3)

sorted_result_df.show()

+--------+----------+---------+----------+
|actor_id|first_name|last_name|film_count|
+--------+----------+---------+----------+
|      17|     HELEN|   VOIGHT|         7|
|     127|     KEVIN|  GARLAND|         5|
|     140|    WHOOPI|     HURT|         5|
+--------+----------+---------+----------+



Stop Spark session:

In [ ]:
spark.stop()